<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/strategy/Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install backtesting

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import ast

class Stock:
  def __init__(self, code):
    self.code = code
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_price(self, start:int='20180501', interval='day'):
    url ='https://api.finance.naver.com/siseJson.naver?symbol=' + self.code + '&requestType=1&startTime=' \
    + start + '&endTime=' + str(datetime.datetime.now().date().strftime("%Y%m%d")) +  '&timeframe=' + interval

    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    b = bs_obj.get_text()
    for i in ['\n', '\t', "\\", ' ']:
      b = b.replace(i,'')

    data = np.array(ast.literal_eval(b)[1:])

    dic = {'Date':list(map(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'), data[:,0])), \
      'Open':np.array(data[:,1], float), 'High':np.array(data[:,2], float), 'Low':np.array(data[:,3], float),\
       'Close':np.array(data[:,4], float), 'Volume':np.array(data[:,5], int)}

    df = pd.DataFrame(data=dic)
    df = df.set_index('Date')
    return df

In [3]:
kospi = Stock('KOSPI')
kospi = kospi.get_price('20000101')

In [4]:
kosdaq = Stock('KOSDAQ')
kosdaq = kosdaq.get_price('20000101')

In [5]:
kosdaq[300:]

,Open,High,Low,Close,Volume
Date,,,,,
2001-03-30,693.30,698.20,667.80,684.30,292828
2001-04-02,684.80,689.60,679.60,682.90,257970
2001-04-03,666.10,666.60,655.00,662.40,289320
2001-04-04,643.70,659.50,641.80,643.40,255877
2001-04-06,676.00,677.70,662.10,665.10,256744
...,...,...,...,...,...
2021-10-18,992.57,997.92,985.14,993.86,1188921
2021-10-19,997.57,1005.35,997.10,1005.35,1375183
2021-10-20,1009.11,1011.55,1001.10,1001.62,1552193


In [6]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

class MarketTiming(Strategy):
    def init(self):
        price = self.data.Close
        self.ma3 = self.I(SMA, price, 3)
        self.ma5 = self.I(SMA, price, 5)
        self.ma10 = self.I(SMA, price, 10)


    def next(self):
        if crossover(self.data.Close, self.ma3) and crossover(self.data.Close, self.ma5) and crossover(self.data.Close, self.ma10):
            self.buy()
        elif crossover(self.ma3, self.data.Close) and crossover(self.ma5, self.data.Close) and crossover(self.ma10, self.data.Close):
            self.sell()


asset = kosdaq[1000:]
bt = Backtest(asset, MarketTiming, commission=.002,
              exclusive_orders=True)
stats = bt.run()

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [7]:
print(stats)

Start                     2004-02-06 00:00:00
End                       2021-10-22 00:00:00
Duration                   6468 days 00:00:00
Exposure Time [%]                     99.2927
Equity Final [$]                      22661.3
Equity Peak [$]                       43607.6
Return [%]                            126.613
Buy & Hold Return [%]                 127.751
Return (Ann.) [%]                     4.81588
Volatility (Ann.) [%]                 22.7358
Sharpe Ratio                         0.211819
Sortino Ratio                        0.340589
Calmar Ratio                        0.0658344
Max. Drawdown [%]                    -73.1514
Avg. Drawdown [%]                    -6.01514
Max. Drawdown Duration     4743 days 00:00:00
Avg. Drawdown Duration      126 days 00:00:00
# Trades                                  209
Win Rate [%]                           45.933
Best Trade [%]                        30.4249
Worst Trade [%]                      -41.7318
Avg. Trade [%]                    

In [8]:
bt.plot()

Row(id='1453', ...)